In [98]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import sys
import pickle
from keras.models import load_model
import matplotlib.pyplot as plt

In [121]:
# read data
df = pd.read_table("OpportunityUCIDataset\dataset\S1-Drill.dat", sep = "\s+", header = None)

In [122]:
# handling nan
for col in df.columns:
    if pd.isna(df.iloc[0, col]):
        df[col][0] = 0
    for row in range(1, len(df[col])):
        if pd.isna(df.iloc[row, col]):
            df[col][row] = df[col][row-1]

C:\Users\DHAIRYA\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\DHAIRYA\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [123]:
# feature scaling
df = abs(df)
df_scaled = MinMaxScaler((0, 1)).fit_transform(df.iloc[:,1:len(df.columns) - 7])
df_scaled = np.around(df_scaled, decimals = 2)
df_scaled = pd.DataFrame(df_scaled)

In [124]:
df

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,0.0,40.0,1031.0,49.0,255.0,958.0,250.0,232.0,969.0,44.0,...,5839.0,2764.0,1815.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,33.0,49.0,1043.0,42.0,244.0,954.0,257.0,242.0,979.0,31.0,...,5840.0,2765.0,1815.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,67.0,17.0,1049.0,84.0,243.0,958.0,278.0,256.0,984.0,2.0,...,5842.0,2766.0,1816.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100.0,23.0,1051.0,98.0,234.0,969.0,295.0,263.0,972.0,10.0,...,5843.0,2767.0,1816.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,133.0,3.0,1045.0,102.0,213.0,975.0,273.0,255.0,1029.0,44.0,...,5841.0,2766.0,1814.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54961,1832015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3926.0,1613.0,869.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54962,1832048.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3073.0,1262.0,680.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54963,1832082.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2202.0,904.0,488.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54964,1832115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1371.0,563.0,304.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [125]:
# labelling
x_train = df_scaled
y_train = df[range(len(df.columns) - 7, len(df.columns))]

In [126]:
data_loco = x_train.iloc[:, :133]
data_loco

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,128,129,130,131,132
0,0.02,0.37,0.02,0.11,0.43,0.10,0.08,0.50,0.02,0.02,...,0.04,0.04,0.11,0.0,0.0,0.0,0.0,0.0,0.0,0.62
1,0.02,0.37,0.01,0.11,0.43,0.10,0.09,0.51,0.01,0.02,...,0.04,0.04,0.11,0.0,0.0,0.0,0.0,0.0,0.0,0.62
2,0.01,0.37,0.03,0.11,0.43,0.11,0.09,0.51,0.00,0.03,...,0.04,0.04,0.11,0.0,0.0,0.0,0.0,0.0,0.0,0.62
3,0.01,0.37,0.03,0.10,0.44,0.12,0.09,0.51,0.00,0.02,...,0.04,0.04,0.11,0.0,0.0,0.0,0.0,0.0,0.0,0.62
4,0.00,0.37,0.03,0.09,0.44,0.11,0.09,0.54,0.02,0.03,...,0.04,0.04,0.11,0.0,0.0,0.0,0.0,0.0,0.0,0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54961,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
54962,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
54963,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
54964,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [127]:
data_loco['Stand'] = [1 if y_train[len(df.columns) - 7][i] == 1 else 0 for i in range(len(data_loco))]
data_loco['Walk'] = [1 if y_train[len(df.columns) - 7][i] == 2 else 0 for i in range(len(data_loco))]
data_loco['Sit'] = [1 if y_train[len(df.columns) - 7][i] == 4 else 0 for i in range(len(data_loco))]
data_loco['Lie'] = [1 if y_train[len(df.columns) - 7][i] == 5 else 0 for i in range(len(data_loco))]
data_loco['Idle'] = [1 if y_train[len(df.columns) - 7][i] == 0 else 0 for i in range(len(data_loco))]

C:\Users\DHAIRYA\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\DHAIRYA\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\DHAIRYA\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [128]:
data_loco.describe()

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,Stand,Walk,Sit,Lie,Idle
count,54966.000000,54966.000000,54966.000000,54966.000000,54966.000000,54966.000000,54966.000000,54966.000000,54966.000000,54966.000000,...,54966.000000,54966.000000,54966.000000,54966.000000,54966.000000,54966.000000,54966.000000,54966.000000,54966.0,54966.000000
mean,0.079334,0.350573,0.106315,0.157960,0.423646,0.052313,0.082082,0.496533,0.058106,0.121013,...,0.039017,0.023712,0.032333,0.039017,0.235655,0.411072,0.436161,0.100717,0.0,0.052050
std,0.075011,0.099769,0.104805,0.070469,0.061523,0.043145,0.063022,0.085661,0.050769,0.087863,...,0.082763,0.051507,0.073967,0.082763,0.226197,0.492033,0.495912,0.300956,0.0,0.222131
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.030000,0.320000,0.030000,0.110000,0.400000,0.020000,0.040000,0.470000,0.020000,0.050000,...,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.0,0.000000
50%,0.060000,0.360000,0.070000,0.150000,0.430000,0.040000,0.070000,0.510000,0.050000,0.100000,...,0.000000,0.000000,0.000000,0.000000,0.130000,0.000000,0.000000,0.000000,0.0,0.000000
75%,0.100000,0.370000,0.160000,0.200000,0.450000,0.070000,0.110000,0.530000,0.080000,0.180000,...,0.020000,0.020000,0.020000,0.020000,0.450000,1.000000,1.000000,0.000000,0.0,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000


In [129]:
# coverting data from 2-D to 3-D for LSTM
xdata = np.array(data_loco)
xdata = [[xdata[i+j][0:133] for j in range(32)] for i in range(len(xdata) -32)]

ydata = np.array(data_loco)
ydata = [ydata[i+32][133:] for i in range(len(data_loco) - 32)]

In [130]:
xdata = np.array(xdata)
ydata = np.array(ydata)

In [131]:
np.save('npy\s1d_x', xdata)
np.save('npy\s1d_y', ydata)